<a href="https://colab.research.google.com/github/Sahar-DataScience/Protein-Location-Prediction/blob/main/IndabaX_bio_workshop_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IndabaX biology workshop

# Install dependencies (hugging face transformer library)

In [ ]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 38.7 MB/s 
     |████████████████████████████████| 636 kB 51.1 MB/s 
     |████████████████████████████████| 895 kB 28.2 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Exploratory Data Analysis and Preprocessing

In [ ]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
df_lin = pd.read_csv("/content/drive/MyDrive/dataset_seq_geo_lin_rbd.csv")

In [ ]:
df_lin.head()

,proteinSequence,continent,country,virusLineage,variantClass,RBD
0,MFAFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,North America,USA,B.1.1.519,vum,RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVAD...
1,MFFFVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSV...,North America,USA,B.1.526,vum,VQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADY...
2,MFGFFVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,North America,USA,B.1.526,vum,RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVAD...
3,MFGFFVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,North America,USA,B.1.526,vum,RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVAD...
4,MFGFFVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,North America,USA,B.1.526,vum,RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVAD...


In [ ]:
df_lin.shape

(17589, 6)

In [ ]:
df_lin["variantClass"].value_counts()

voc    11437
vum     6044
voi      108
Name: variantClass, dtype: int64

In [ ]:
# Later when using the pretrained model the protein sequences have to be tokenized using a single space that's why we need to add spaces between amino acids.
df_lin['RBD'] = df_lin['RBD'].apply(lambda seq: " ".join(seq))

In [ ]:
df_lin['RBD']

0        R V Q P T E S I V R F P N I T N L C P F G E V ...
1        V Q P T E S I V R F P N I T N L C P F G E V F ...
2        R V Q P T E S I V R F P N I T N L C P F G E V ...
3        R V Q P T E S I V R F P N I T N L C P F G E V ...
4        R V Q P T E S I V R F P N I T N L C P F G E V ...
                               ...                        
17584    P T E S I V R F P N I T N L C P F G E V F N A ...
17585    P T E S I V R F P N I T N L C P F G E V F N A ...
17586    P T E S I V R F P N I T N L C P F G E V F N A ...
17587    P T E S I V R F P N I T N L C P F G E V F N A ...
17588    P T E S I V R F P N I T N L C P F G E V F N A ...
Name: RBD, Length: 17589, dtype: object

In [ ]:
# The rare amino acids "U,Z,O,B" were mapped to "X"
df_lin["RBD"] = df_lin["RBD"].str.replace(r"[UZOB]", 'X', regex=True)

In [ ]:
targets = df_lin['variantClass'].unique()

In [ ]:
# a dictionary mapping between labels and classes
label_dict = {}
for id, lin in enumerate(targets):
  label_dict[lin] = id

In [ ]:
label_dict

{'voc': 1, 'voi': 2, 'vum': 0}

In [ ]:
df_lin["label"] = df_lin["variantClass"].replace(label_dict)

In [ ]:
df_lin.head()

,proteinSequence,continent,country,virusLineage,variantClass,RBD,label
0,MFAFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,North America,USA,B.1.1.519,vum,R V Q P T E S I V R F P N I T N L C P F G E V ...,0
1,MFFFVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSV...,North America,USA,B.1.526,vum,V Q P T E S I V R F P N I T N L C P F G E V F ...,0
2,MFGFFVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,North America,USA,B.1.526,vum,R V Q P T E S I V R F P N I T N L C P F G E V ...,0
3,MFGFFVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,North America,USA,B.1.526,vum,R V Q P T E S I V R F P N I T N L C P F G E V ...,0
4,MFGFFVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,North America,USA,B.1.526,vum,R V Q P T E S I V R F P N I T N L C P F G E V ...,0


# Training/Validation Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df_lin.index.values,
    df_lin.label.values,
    test_size=0.15,
    random_state=7,
    stratify=df_lin.label.values,
)

In [ ]:
df_lin["data_type"] = ["unk"] * df_lin.shape[0]

In [ ]:
df_lin.loc[X_train, "data_type"] = "train"
df_lin.loc[X_val, "data_type"] = "val"

In [ ]:
df_lin.groupby(["variantClass", "label", 'data_type']).count()

proteinSequence  continent  ...  virusLineage   RBD
variantClass label data_type                              ...                    
voc          1     train                 9721       9721  ...          9721  9721
                   val                   1716       1716  ...          1716  1716
voi          2     train                   92         92  ...            92    92
                   val                     16         16  ...            16    16
vum          0     train                 5137       5137  ...          5137  5137
                   val                    907        907  ...           907   907

[6 rows x 5 columns]

# Loading Tokenizer and Encoding our Data

In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [ ]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)

Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/361 [00:00<?, ?B/s]

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    df_lin[df_lin.data_type=="train"].RBD.values,
    add_special_tokens=True,
    padding=True,
    return_attention_mask=True,
    return_tensors='pt',
)

encoded_data_val = tokenizer.batch_encode_plus(
    df_lin[df_lin.data_type=="val"].RBD.values,
    add_special_tokens=True,
    padding=True,
    return_attention_mask=True,
    return_tensors='pt',
)

In [ ]:
encoded_data_train

{'input_ids': tensor([[ 2, 13,  8,  ..., 17, 19,  3],
        [ 2,  8, 18,  ..., 19, 17,  3],
        [ 2, 13,  8,  ..., 17, 19,  3],
        ...,
        [ 2, 16, 15,  ..., 19, 17,  3],
        [ 2, 16, 15,  ..., 19, 17,  3],
        [ 2, 16, 15,  ..., 19, 17,  3]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
input_ids_train = encoded_data_train["input_ids"]
attention_masks_train = encoded_data_train["attention_mask"]
labels_train = torch.tensor(df_lin[df_lin.data_type=="train"].label.values)

input_ids_val = encoded_data_val["input_ids"]
attention_masks_val = encoded_data_val["attention_mask"]
labels_val = torch.tensor(df_lin[df_lin.data_type=="val"].label.values)

In [ ]:
train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
val_dataset = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(train_dataset), len(val_dataset)

(14950, 2639)

# Creating Data Loaders

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=1,
)
# For validation the order doesn't matter, so we'll just read them sequentially.
val_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=2,
)

# Setting up BERT Pretrained Model

In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top.
model = BertForSequenceClassification.from_pretrained("Rostlab/prot_bert", 
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False
                                                      )

Downloading:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [ ]:
for param in model.bert.parameters():
    param.requires_grad = False

# Setting Up Optimizer and Scheduler

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr=2e-5, # 0.00002
    eps=1e-8,
)

In [ ]:
epochs = 3
num_training_steps = epochs * len(train_dataloader)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

# Defining our Performance Metrics

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

In [ ]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=-1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average="weighted")

In [ ]:
def accuracy_per_class(preds, labels):
  inverse_label_dict = {v: k for k, v in label_dict.items()}
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f"class: {inverse_label_dict[label]}")
    print(f"Accuracy: {len(y_preds[y_preds==label])} / {len(y_true)}\n")

# Creating our Training Loop

In [ ]:
import random

seed_val = 7
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [ ]:
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):
  # Put the model into eval mode.
    model.eval()
    
    total_val_loss = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        # Unpack this validation batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            # forward pass       
            outputs = model(**inputs)   
        loss = outputs.loss
        logits = outputs.logits
        # Accumulate the validation loss.
        total_val_loss += loss.item()
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    # Calculate the average loss over all of the batches.
    val_loss_avg = total_val_loss/len(val_dataloader) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return val_loss_avg, predictions, true_vals

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# our training loop
# Put the model into training mode.
model.train()
for epoch in tqdm(range(1, epochs+1)):
  progress_bar = tqdm(train_dataloader,
                    desc="Epoch{:1d}".format(epoch),
                    leave=False,
                    disable=False,)
  # Reset the total loss for this epoch.
  total_train_loss = 0.0
  for batch in train_dataloader:
    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using the 
    # `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    batch = tuple(b.to(device) for b in batch)
    inputs = {'input_ids':      batch[0],
              'attention_mask': batch[1],
              'labels':         batch[2],
              }
    # Clear out the gradients calculated in the previous pass
    optimizer.zero_grad()
    # foraward pass to evaluate the training batch
    outputs = model(**inputs)
    loss = outputs.loss
    # Accumulate the training loss.
    total_train_loss += loss.item()
    # Perform a backward pass to calculate the gradients.
    loss.backward()
    # Clip the norm of the gradients to 1.0.
    # This is to help prevent the "exploding gradients" problem.
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    # Update parameters
    optimizer.step()
    # Update the learning rate
    scheduler.step()
    progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item())})
    progress_bar.update(1)

    

  tqdm.write(f"\nEpoch {epoch}")
  # Calculate the average loss over all of the batches.
  train_loss_avg = total_train_loss / len(train_dataloader)
  tqdm.write(f"Training loss: {train_loss_avg}")

  val_loss, predictions, true_vals = evaluate(val_dataloader)
  # Report accuracy
  val_f1 = f1_score_func(predictions, true_vals)
  tqdm.write(f"validation loss: {val_loss}")
  tqdm.write(f"F1 score (weighted): {val_f1}")
  torch.save(model.state_dict(), f"lin_classifier_{epoch}.pt")

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch1:   0%|          | 0/24902 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Loading and Evaluating our Model

In [ ]:
model = BertForSequenceClassification.from_pretrained("Rostlab/prot_bert",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/lin_classifier_3.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
model.to(device)
pass

In [ ]:
_, predictions, true_vals = evaluate(val_dataloader)

  0%|          | 0/1320 [00:00<?, ?it/s]

In [ ]:
f1_score_func(predictions, true_vals)

0.512432909344901

In [ ]:
accuracy_per_class(predictions, true_vals)

class: vum
Accuracy: 0 / 907

class: voc
Accuracy: 1716 / 1716

class: voi
Accuracy: 0 / 16



# Next step

Try to improve th model
- find a solution to class imbalance (class weighting, resampling data, data augmentation...)
- try other methods of fine tuning (gradually unfreezing the model)

  ```
  def freeze(model, frozen_layers):
      modules = [model.bert.encoder.layer[:frozen_layers]] 
      for module in modules:
          for param in module.parameters():
              param.requires_grad = False
  ```



